In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from weldx import Q_, WeldxFile
import time

## File access
- open the file and copy contents into local `data`
- open interactive weldx tree view oif the file contents
- print some experiment metadata

In [ ]:
filename = "./data/WID594.wx"

In [ ]:
with WeldxFile(filename, "r", memmap=False) as wx_file:
    data = wx_file.copy()

In [ ]:
data

In [ ]:
print(f"WID={data['WID']}")
print(data["workpiece"]["condition"])
print(data["observations"])

## welding current & voltage plots

In [ ]:
current = data["welding_current"]
voltage = data["welding_voltage"]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
current.plot(axes=ax1)
voltage.plot(axes=ax2)

## Coordinate Systems
- Extract the `CoordinateSystemManager` from the data
- Show the experiment in the (original) `USER_FRAME` System (movement "right to left")
- Show the experiment in the `WELD_FRAME` System
- get the welding TCP movement in the `WELD_FRAME` System for further analysis

In [ ]:
csm = data["CSM"]

In [ ]:
csm.plot(
    backend="k3d",
    reference_system="USER_FRAME",
    coordinate_systems=["weld_tcp", "WELD_FRAME", "USER_FRAME"],
    data_sets=["scan_1"],
    colors={
        "scan_0": [170, 170, 170],
        "scan_1": [170, 170, 170],
    },
)

In [ ]:
plot = csm.plot(
    backend="k3d",
    reference_system="WELD_FRAME",
    coordinate_systems=["weld_tcp", "WELD_FRAME"],
    data_sets=["scan_1"],
    colors={
        "scan_0": [170, 170, 170],
        "scan_1": [170, 170, 170],
    },
)
plot.plot.grid = [-50, -60, -5, 250, 60, 20]
display(plot)

time.sleep(0.1)
plot.plot.camera = [85.9, 0.99, 450.5, 85.9, 0.99, -11.78, 0, 0, 1]
plot.plot.camera_fov = 20

In [ ]:
weld_lcs = csm.get_cs("weld_tcp", "WELD_FRAME")

## Combined position and time plot
- plot the welding voltage and current with both time and x-position along the weld seam

In [ ]:
for signal in [current, voltage]:
    fig, ax = plt.subplots(layout="constrained")
    x = signal.time.as_quantity("s")
    y = signal.data
    ax.plot(x, y)
    ax.set_xlabel("time in s")
    ax.set_ylabel(f"{signal.units}")
    ax.set_xlim([0, 15])

    def time2pos(x):
        """Get position at specific time"""
        coords = weld_lcs.interp_time(Q_(x, "s")).coordinates
        return coords.sel(c="x").data.m

    def pos2time(x):
        """Get time based on x-position."""
        x_coords = weld_lcs.coordinates.sel(c="x")
        distance = np.abs(x_coords - xr.DataArray(data=Q_(x, "mm")))
        indx = distance.argmin(dim="time")
        return x_coords.isel(time=indx).data.m

    secax = ax.secondary_xaxis("top", functions=(time2pos, pos2time))
    secax.set_xlabel("x position in mm")
    plt.show()